In [1]:
from dictionary_learning import CrossCoder
from torch.nn.functional import cosine_similarity
import torch as th
import plotly.express as px
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd

th.set_grad_enabled(False)
exp_name = "eval_crosscoder"
# %load_ext autoreload
# %autoreload 2

In [2]:
data = th.load("/dlabscratch1/jminder/repositories/representation-structure-comparison/high_threshold_indices.pt")

/tmp/ipykernel_2458967/77499533.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = th.load("/dlabscratch1/jminder/repositories/representation-structure-comparison/hi

In [2]:
!export TOKENIZERS_PARALLELISM=false

Using runai image conda


In [4]:
import sys
sys.path.append("..")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [2]:
crosscoder_path = "/dlabscratch1/jminder/repositories/representation-structure-comparison/checkpoints/l13-mu4.1e-02-lr1e-04/ae_final.pt"
extra_args = []
exp_id = ""
device = "cuda"
seed = 42
base_model = "google/gemma-2-2b"
instruct_model = "google/gemma-2-2b-it"
layer = 13
activation_dir = Path(
    "/dlabscratch1/jminder/repositories/representation-structure-comparison/activations"
)
validation_size = 10**6
model_batch_size = 64
workers = 12
SEQ_LEN = 1024
n = 100
crosscoder_batch_size = 2048

In [5]:
data_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data"
)
df = pd.read_csv(data_path / "feature_df.csv")
# Filter for IT only and Base only features that are not dead
selected_features = df[(df['tag'].isin(['IT only', 'Base only'])) & (df['dead'] == False)]

# Get the indices of the selected features in the dataframe
selected_indices = selected_features.index.tolist()

print(f"Number of selected features: {len(selected_indices)}")
print(f"First 10 selected indices: {selected_indices[:10]}")

Number of selected features: 4532
First 10 selected indices: [55, 60, 78, 82, 95, 112, 119, 130, 140, 221]


## Feature identification

In [6]:
base_only_features = df[df['tag'] == 'Base only'].sort_values(by='lmsys_freq', ascending=False).head(50)
base_only_features

,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
39490,39490,False,0.000333,False,0.029302,0.013326,False,1.469457,1.479005,0.786849,0.583952,0.008512,0.992711,Base only,1.693535,0.918406,0.728849,0.729923,-0.438501
53366,53366,False,0.000284,False,0.027410,0.012450,False,2.526684,2.546929,0.935645,0.662858,0.010607,0.991999,Base only,1.547133,0.742314,0.760100,0.500804,-0.647388
56811,56811,False,0.027413,False,0.022537,0.025226,False,0.842609,0.807367,0.866173,0.496897,0.006603,0.993355,Base only,1.262340,0.557609,0.779137,0.723344,-0.361435
21528,21528,False,0.000174,False,0.021098,0.009558,False,1.552147,1.558374,0.937205,0.572815,0.012552,0.989044,Base only,1.548867,0.845972,0.726906,0.630517,-0.448959
22372,22372,False,0.008750,False,0.020236,0.013901,False,1.257422,1.221356,1.325260,0.541626,0.011767,0.989137,Base only,1.351881,0.699046,0.741454,0.796100,-0.430285
45280,45280,False,0.001849,False,0.019509,0.009770,False,4.686795,4.287578,8.112745,1.024995,0.012271,0.994014,Base only,1.383188,0.616033,0.777314,0.572744,-0.589276
66000,66000,False,0.000779,False,0.016765,0.007949,False,1.342104,1.360509,1.019796,0.547868,0.014253,0.986992,Base only,1.555652,0.795350,0.744368,0.716503,-0.453803
23342,23342,False,0.000549,False,0.015685,0.007338,False,1.582841,1.608163,0.994586,0.555924,0.028621,0.974258,Base only,1.725707,0.812282,0.764652,0.762075,-0.475051
58252,58252,False,0.004775,False,0.015581,0.009622,False,1.987252,2.108041,1.666740,0.675037,0.013287,0.990158,Base only,1.235030,0.748570,0.696943,0.723741,-0.486860
63299,63299,False,0.000244,False,0.015198,0.006951,False,1.479514,1.490753,0.910005,0.592878,0.010568,0.991088,Base only,1.585108,0.847075,0.732802,0.623571,-0.438642


In [20]:
df.iloc[593]

Unnamed: 0                    593
fw_dead                     False
fw_freq                  0.001531
lmsys_dead                  False
lmsys_freq               0.001429
freq                     0.001485
dead                        False
avg_activation            3.63542
lmsys_avg_activation     3.992857
fw_avg_activation        3.364052
dec_base_norm            0.700494
dec_instruct_norm        0.027105
dec_norm_diff            0.980653
tag                     Base only
enc_base_norm            1.852539
enc_instruct_norm        0.606914
enc_norm_diff            0.836194
dec_cos_sim              0.674656
enc_cos_sim              -0.07113
Name: 593, dtype: object

In [23]:
import json
it_only_features = th.load(data_path / "only_it_decoder_feature_indices.pt", weights_only=True)
dead_scale_features_1e_4 = json.load(
    open(save_path.parent / "scalar" / "dead_feature_indices_1e-4.json", "r")
)
dead_scale_features_1e_4 = [it_only_features[i].item() for i in dead_scale_features_1e_4]
print(len(dead_scale_features_1e_4))
print(f"not deads: {len([i for i in dead_scale_features_1e_4 if not df.iloc[i]['dead']])}")
dead_scale_features_all = json.load(
    open(save_path.parent / "scalar" / "dead_feature_indices_1e-2.json", "r")
)
dead_scale_features_all = [it_only_features[i] for i in dead_scale_features_all]
with pd.option_context('display.max_rows', None):
    display(df[df.index.isin(dead_scale_features_1e_4) & (df['dead'] == False)].style.format({'lmsys_freq': '{:.1e}', 'fw_freq': '{:.1e}', 'freq': '{:.1e}'})) # .sort_values(by='lmsys_freq', ascending=False)

70
not deads: 70


,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
771,771,False,6.2e-03,False,1.1e-03,3.9e-03,False,1.909597,1.950318,1.903743,0.019161,0.601516,0.015928,IT only,0.932539,1.535208,0.303717,0.518771,-0.523397
883,883,False,2.5e-03,False,3.6e-03,3.0e-03,False,3.717293,3.935713,3.455262,0.016195,0.695830,0.011637,IT only,0.911130,1.705567,0.267105,0.612343,-0.028956
1045,1045,False,8.2e-04,False,3.4e-03,2.0e-03,False,2.844598,3.318568,1.259410,0.036045,0.798618,0.022567,IT only,1.015253,1.621810,0.313000,0.547398,-0.626637
1353,1353,False,3.1e-04,False,7.2e-04,5.0e-04,False,5.033971,5.494558,4.174287,0.051135,0.809860,0.031570,IT only,0.951458,1.758868,0.270474,0.509238,-0.093958
1742,1742,False,2.2e-03,False,9.6e-04,1.7e-03,False,2.415570,2.296070,2.457325,0.035891,0.584446,0.030705,IT only,0.849956,1.651648,0.257305,0.677590,-0.223040
2412,2412,False,2.0e-03,False,1.5e-03,1.8e-03,False,3.363636,3.214202,3.454763,0.031885,0.653488,0.024396,IT only,0.849247,1.657459,0.256189,0.697360,-0.092483
3129,3129,False,5.6e-03,False,3.5e-03,4.7e-03,False,2.808331,2.736047,2.845446,0.012027,0.638428,0.009419,IT only,0.873279,1.563975,0.279186,0.444073,-0.273691
3857,3857,False,3.7e-03,False,1.2e-03,2.6e-03,False,2.698874,2.617466,2.720279,0.022227,0.664758,0.016718,IT only,0.825620,1.507581,0.273823,0.491178,-0.380666
4194,4194,False,1.8e-03,False,1.2e-03,1.6e-03,False,2.712537,2.548630,2.801038,0.019319,0.659032,0.014657,IT only,0.824485,1.631766,0.252636,0.376209,-0.042035
5406,5406,False,8.8e-05,False,3.7e-04,2.1e-04,False,7.158593,7.868626,4.727653,0.112356,0.940421,0.059737,IT only,1.088786,1.564219,0.348029,0.230806,-0.107499


In [ ]:
df[df.index.isin(dead_scale_features_all)] # .sort_values(by='lmsys_freq', ascending=False)

In [21]:
import json
fossil_features = json.load(open("/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data/lmsys_it_fossil_indices.json", "r"))
print(fossil_features)
# show df for fossil features
df[df.index.isin(fossil_features)].sort_values(by='lmsys_freq', ascending=False)

[1697, 1746, 2667, 3152, 3792, 3913, 4179, 4874, 4940, 6911, 7269, 11743, 12121, 14538, 21710, 24731, 27129, 28200, 29706, 30133, 30142, 30859, 34253, 35465, 38917, 39351, 44612, 45142, 50764, 51124, 51224, 51492, 54708, 55240, 55641, 56484, 56981, 57713, 59641, 60186, 61285, 62571, 63664, 67357, 69536]


,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
51224,51224,False,0.002244,False,0.009417,0.005461,False,1.356139,1.434295,1.089368,0.015165,0.620764,0.012215,IT only,0.781824,1.287962,0.303512,0.678796,-0.315993
14538,14538,False,0.004970,False,0.008652,0.006621,False,2.033081,2.228742,1.756093,0.010553,0.693029,0.007614,IT only,0.925675,1.212323,0.381777,0.538052,-0.430217
60186,60186,False,0.009121,False,0.007596,0.008437,False,1.697013,1.558748,1.790660,0.014414,0.650730,0.011075,IT only,0.738039,1.112684,0.331648,0.714217,-0.556696
3792,3792,False,0.002327,False,0.007561,0.004674,False,2.004497,2.098311,1.756544,0.019425,0.744636,0.013043,IT only,0.860620,1.198700,0.358980,0.721904,-0.359496
21710,21710,False,0.008610,False,0.007376,0.008057,False,1.645800,1.688388,1.616128,0.012726,0.630944,0.010085,IT only,0.861562,1.136083,0.379181,0.702146,-0.371451
24731,24731,False,0.007945,False,0.007158,0.007592,False,1.914850,1.937446,1.898292,0.014258,0.655771,0.010871,IT only,0.954068,1.188913,0.401235,0.724845,-0.503147
4940,4940,False,0.005053,False,0.005609,0.005302,False,1.772930,1.780608,1.765998,0.015082,0.702384,0.010736,IT only,1.004360,1.340401,0.374649,0.628947,-0.418253
62571,62571,False,0.008807,False,0.004892,0.007051,False,1.622470,1.483220,1.685378,0.019504,0.614624,0.015866,IT only,0.711940,1.189054,0.299372,0.808625,-0.551995
1746,1746,False,0.006107,False,0.004677,0.005466,False,1.541892,1.618612,1.494107,0.012829,0.657272,0.009760,IT only,0.839238,1.219396,0.344121,0.622194,-0.353978
11743,11743,False,0.007542,False,0.004623,0.006233,False,1.735984,1.816740,1.695722,0.013883,0.716744,0.009685,IT only,0.840073,1.131233,0.371309,0.678448,-0.370328


## Offline dashboard

In [ ]:
load_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/results-runai/max_activating_examples"
)
save_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/results/max_activating_examples"
)
# generate the max activating examples
# max_activating_ex_mini = th.load(save_path / "max_activating_examples_mini_final_cleaned.pt")
# max_activating_examples_base = th.load(save_path / "max_activating_examples_base.pt")
max_activating_examples_chat = th.load(load_path / "chat/chat_final.pt")
# max_activating_examples_base = th.load(load_path / "base/base_final.pt")

In [17]:
from neel.utils import create_html
create_html(max_activating_examples_chat[1045][-1][1], max_activating_examples_chat[1045][-1][-1])


In [10]:
from feature_dashboard import FeatureCentricDashboard
from transformers import AutoTokenizer
dashboard = FeatureCentricDashboard(max_activation_examples=max_activating_examples_chat, tokenizer=AutoTokenizer.from_pretrained(instruct_model))
dashboard.display()

## Online dashboard

In [5]:
!pip install git+https://github.com/jkminder/dictionary_learning.git@crosscoders --no-deps

  Cloning https://github.com/jkminder/dictionary_learning.git (to revision crosscoders) to /tmp/pip-req-build-w8woeth4
  Running command git clone --filter=blob:none --quiet https://github.com/jkminder/dictionary_learning.git /tmp/pip-req-build-w8woeth4
  Running command git checkout -b crosscoders --track origin/crosscoders
  Switched to a new branch 'crosscoders'
  Branch 'crosscoders' set up to track remote branch 'crosscoders' from 'origin'.
  Resolved https://github.com/jkminder/dictionary_learning.git to commit 0fad3eb560c0dcfde567ea61ee2d244320df331e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dictionary_learning: filename=dictionary_learning-0.1.dev280+g0fad3eb-py3-none-any.whl size=46889 sha256=505e5188fdaa5fb10bd5b58e47b345bb6819614bea3d110213cbba7085befda7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ac_jx4uc/wheels/67/1a/d8/3f6c1ffabbe81b85baacbfa01490d54b877f

In [7]:
import sys
sys.path.append("..")

device="cuda:1"

In [8]:
from feature_dashboard import OnlineFeatureCentricDashboard
from nnterp import load_model

b_model = load_model(
    base_model, attn_implementation="eager", torch_dtype="bfloat16", device_map=device
)
it_model = load_model(
    instruct_model, attn_implementation="eager", torch_dtype="bfloat16", device_map=device
)
crosscoder = CrossCoder.from_pretrained(crosscoder_path, device=device)


RuntimeError: duplicate registrations for aten.linspace.Tensor_Tensor

In [34]:
crosscoder.encoder.bias[1045]

tensor(-0.2021, device='cuda:0', dtype=torch.float32, requires_grad=True)

In [6]:
dashboard = OnlineFeatureCentricDashboard(b_model, it_model, crosscoder, 13)
dashboard.display()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([12, 2304])
(2,)


torch.Size([86, 2304])
(1045,)


# SAE LENS (PAIN)

In [12]:
from sae_lens import SAE, SAEConfig
from argparse import ArgumentParser

parser = ArgumentParser()
parser.add_argument("--base-device", type=str, default=device)
parser.add_argument("--instruct-device", type=str, default=device)
args = parser.parse_args(extra_args)

base_config_dict = {
    "architecture": "standard",
    "d_in": activation_dim,
    "d_sae": dict_size,
    "dtype": "float32",
    "model_name": base_model,
    "hook_name": f"blocks.{layer}.hook_resid_post",
    "hook_layer": layer,
    "hook_head_index": None,
    "activation_fn_str": "relu",
    "finetuning_scaling_factor": False,
    "sae_lens_training_version": None,
    "prepend_bos": True,
    "dataset_path": None,
    "context_size": 1024,
    "dataset_trust_remote_code": False,
    "apply_b_dec_to_input": False,
    "normalize_activations": None,
    "device": "cpu",
    # "device": args.base_device,
}
base_config = SAEConfig.from_dict(base_config_dict)

it_config_dict = base_config_dict.copy()
it_config_dict["model_name"] = instruct_model
it_config_dict["device"] = args.instruct_device
it_config = SAEConfig.from_dict(it_config_dict)


def gen_state_dict(model_idx):
    return {
        "b_enc": crosscoder.encoder.bias,
        "W_enc": crosscoder.encoder.weight[model_idx],
        "b_dec": crosscoder.decoder.bias[model_idx],
        "W_dec": crosscoder.decoder.weight[model_idx],
    }

## Base Visualization

In [13]:
base_sae = SAE(base_config)
base_sae.load_state_dict(gen_state_dict(0))
base_sae.fold_W_dec_norm()


In [14]:
from transformer_lens import HookedTransformer, utils
from sae_dashboard.sae_vis_data import SaeVisConfig
from sae_dashboard.sae_vis_runner import SaeVisRunner
from datasets import load_dataset, load_from_disk

# Load model and SAE
model = HookedTransformer.from_pretrained_no_processing(
    base_model, device=args.base_device, dtype="float16", n_devices=2
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [15]:

fineweb = load_dataset(
    "HuggingFaceFW/fineweb",
    name="sample-10BT",
    split="train",
    cache_dir=Path("/dlabscratch1/cdumas/.cache/huggingface/datasets/"),
)

# select 300 random samples
indices = th.randperm(len(fineweb))[:300]
fineweb = fineweb.select(indices)
fineweb_tokenized_data = utils.tokenize_and_concatenate(fineweb, model.tokenizer, max_length=SEQ_LEN)  # type: ignore
lmsys = load_from_disk("/dlabscratch1/public/datasets/lmsys-chat-1m-formatted/")
indices = th.randperm(len(lmsys))[:300]
lmsys = lmsys.select(indices)
lmsys_tokenized_data = utils.tokenize_and_concatenate(lmsys, model.tokenizer, max_length=SEQ_LEN)  # type: ignore


Resolving data files:   0%|          | 0/23781 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/102 [00:00<?, ?it/s]

Map (num_proc=10):   0%|          | 0/300 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/300 [00:00<?, ? examples/s]

In [16]:
# concatenate the two datasets
from datasets import concatenate_datasets
tokenized_data = concatenate_datasets([fineweb_tokenized_data, lmsys_tokenized_data])


In [17]:
base_features = th.load("/dlabscratch1/cdumas/representation-structure-comparison/results/eval_crosscoder/checkpoints_l13-mu4.0e-02-lr1e-04_ae_90000.pt/data/only_base_decoder_feature_indices.pt", weights_only=True).tolist()
it_features = th.load("/dlabscratch1/cdumas/representation-structure-comparison/results/eval_crosscoder/checkpoints_l13-mu4.0e-02-lr1e-04_ae_90000.pt/data/only_it_decoder_feature_indices.pt", weights_only=True).tolist()
shared_features = th.load("/dlabscratch1/cdumas/representation-structure-comparison/results/eval_crosscoder/checkpoints_l13-mu4.0e-02-lr1e-04_ae_90000.pt/data/shared_decoder_feature_indices.pt", weights_only=True).tolist()
all_features = base_features[:10] + it_features[:10] + shared_features[:10]

In [21]:
base_features[:10]

[78, 95, 222, 263, 377, 418, 585, 593, 603, 652]

In [19]:
from sae_dashboard.sae_vis_data import SaeVisData


# Configure visualization
config = SaeVisConfig(
    hook_point=base_sae.cfg.hook_name,
    features=all_features,
    device=args.base_device,
    dtype="bfloat16",
    minibatch_size_features=64,
    minibatch_size_tokens=16,
    verbose=True,
)

In [6]:
import ipywidgets as widgets
from IPython.display import display

def check_feature_type(feature):
    if feature in base_features:
        return "Base"
    elif feature in it_features:
        return "IT"
    elif feature in shared_features:
        return "Shared"
    else:
        return "Not found in any list"

feature_input = widgets.IntText(
    value=0,
    description='Feature:',
    disabled=False
)

output = widgets.Output()

def on_value_change(change):
    with output:
        output.clear_output()
        feature_type = check_feature_type(change['new'])
        print(f"Feature {change['new']} is: {feature_type}")

feature_input.observe(on_value_change, names='value')

display(feature_input, output)


IntText(value=0, description='Feature:')

Output()

In [20]:
data = SaeVisRunner(config).run(encoder=base_sae, model=model, tokens=tokenized_data['tokens'][:16])

# Save feature-centric visualization


Forward passes to cache data for vis:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/30 [00:00<?, ?it/s]

Feature batches:   0%|          | 0/1 [00:00<?, ?it/s]

┏━━━━━━┳━━━━━━┳━━━━━━━┓
┃ Task ┃ Time ┃ Pct % ┃
┡━━━━━━╇━━━━━━╇━━━━━━━┩
└──────┴──────┴───────┘

Saving feature-centric vis:   0%|          | 0/30 [00:00<?, ?it/s]

In [22]:
from sae_dashboard.data_writing_fns import save_feature_centric_vis
save_feature_centric_vis(sae_vis_data=data, filename="feature_dashboard_base.html", separate_files=True)

Saving feature-centric vis:   0%|          | 0/30 [00:00<?, ?it/s]

## IT visualization

In [ ]:
# it_sae = SAE(it_config)
# it_sae.load_state_dict(gen_state_dict(1))
